In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os

from protoplast.genomics.anndata import read_h5ad
from protoplast.patches.anndata_read_h5ad_backed import apply_read_h5ad_backed_patch
from protoplast.patches.anndata_remote import apply_file_backing_patch
from protoplast.patches.daft_flotilla import apply_flotilla_patches

os.environ["MAX_WORKERS"] = "4"
apply_flotilla_patches()
apply_file_backing_patch()
apply_read_h5ad_backed_patch()

✓ Applied AnnDataFileManager patch
✓ Applied daft flotilla patch: start_ray_workers now creates one worker per CPU
✓ Applied AnnDataFileManager patch


In [3]:
%%time
import daft

daft.context.set_runner_ray()
daft.context.set_execution_config(native_parquet_writer=True)

home_path = os.environ["HOME"]
file_path = f"{home_path}/Downloads/plate7_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad"

df = read_h5ad(file_path, 25000).write_parquet(f"{home_path}/Softwares/protoplast/plate7-write-25kbatch-4workers")

2025-08-15 19:26:58,996	INFO worker.py:1747 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2025-08-15 19:26:59,011	INFO worker.py:1927 -- Connected to Ray cluster.


PhysicalScan->PhysicalWrite:   0%|          | 0/1 [00:00<?, ?it/s]

(RaySwordfishActor pid=80459) ✓ Applied AnnDataFileManager patch
CPU times: user 7.16 s, sys: 3.75 s, total: 10.9 s
Wall time: 13min 23s


In [4]:
df = daft.read_parquet(f"{home_path}/Softwares/protoplast/plate7-write-25kbatch-4workers")

In [5]:
%%time
df.count().show()

countUInt64
5692117


CPU times: user 409 ms, sys: 247 ms, total: 655 ms
Wall time: 864 ms


In [6]:
%%time
df.select("C1orf112").mean().show()

C1orf112Float64
0.04183610421219381


CPU times: user 405 ms, sys: 238 ms, total: 643 ms
Wall time: 929 ms


In [7]:
%%time
df.select("C1orf112").max().show()

C1orf112Float32
29


CPU times: user 397 ms, sys: 237 ms, total: 634 ms
Wall time: 840 ms
